In [ ]:
import pandas as pd
from config import AGGREGATED_DATA_PATH

In [ ]:
btc_data = pd.read_hdf(f"{AGGREGATED_DATA_PATH}BTCUSDT-1s.h5")
alt_data = pd.read_hdf(f"{AGGREGATED_DATA_PATH}ANTUSDT-1s.h5")

In [ ]:
def prepare_data(latest: int, coin_name,btc_data,alt_data):
    df = pd.DataFrame({"btc":btc_data[-latest:]["Close"],coin_name:alt_data[-latest:]["Close"]})
    df["btc_return"] = df["btc"]/df["btc"].shift() - 1
    df[f"{coin_name}_return"] = df[f"{coin_name}"]/df[f"{coin_name}"].shift() - 1
    for i in range(0,8,1):
        df[f"btc_fwd_{i}"] = df["btc_return"].shift(-i)
        df[f"{coin_name}_fwd_{i}"] = df[f"{coin_name}_return"].shift(-i)
    df.dropna(inplace=True)
    return df

In [ ]:
df = prepare_data(
    latest=5_000_000,
    coin_name="ant",
    btc_data=btc_data,
    alt_data=alt_data
)

In [ ]:
def plot_drop_correlation(df:pd.DataFrame,delay:int,coin_name:str,drop_ratio:float):
    list = []
    for num in range(200):
        print(num, end = " ")
        step = -0.00001
        th = num * step
        ratio = len(df[(df["btc_return"] < th) & (df[f"{coin_name}_fwd_{delay}"] < th * drop_ratio)]) / len(df[(df["btc_return"] < th)])\
            if  len(df[df["btc_return"] >0]) else 0
        list.append(ratio)
        
    pd.DataFrame(list).plot()

In [ ]:
def plot_drop_event_counts(df:pd.DataFrame):
    list = []
    for num in range(200):
        print(num, end = " ")
        step = -0.00001
        th = num * step
        list.append(len(df[df["btc_return"]<th]))
    pd.DataFrame(list).plot(logy=True)

In [ ]:
plot_drop_correlation(df=df,delay=1,coin_name="ant",drop_ratio=0.1)